In [1]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2
import numpy as np

INPUT_CSV = '../data/processed/train_detection.csv'
OUTPUT_DIR = '../data/yolo_dataset'

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

for split in ['train', 'val']:
    os.makedirs(os.path.join(OUTPUT_DIR, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, 'labels', split), exist_ok=True)

if not os.path.exists(INPUT_CSV):
    print("ERROR: No existe el CSV. Ejecuta el Notebook 01 primero.")
else:
    df = pd.read_csv(INPUT_CSV)
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

    def process_data_with_zoom(dataframe, split_name):
        print(f"Procesando {split_name} (Generando Zoom Artificial)...")
        count = 0
        
        for _, row in tqdm(dataframe.iterrows(), total=len(dataframe)):
            img_path = row['image_path']
            
            if not os.path.exists(img_path):
                alt_path = os.path.abspath(os.path.join('..', img_path))
                if os.path.exists(alt_path):
                    img_path = alt_path
                else:
                    filename = os.path.basename(img_path)
                    alt_path_2 = os.path.join('../data/raw', filename)
                    if os.path.exists(alt_path_2):
                        img_path = alt_path_2
                    else:
                        continue
            
            img = cv2.imread(img_path)
            if img is None: continue
            h_img, w_img = img.shape[:2]
            
            if 'xmin' in row and not pd.isna(row['xmin']):
                x1 = int(row['xmin'])
                y1 = int(row['ymin'])
                x2 = int(row['xmax'])
                y2 = int(row['ymax'])
            elif 'x' in row and not pd.isna(row['x']):
                x1 = int(row['x'])
                y1 = int(row['y'])
                x2 = int(row['x'] + row['w'])
                y2 = int(row['y'] + row['h'])
            else:
                continue 

            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w_img, x2), min(h_img, y2)
            
            if x2 <= x1 or y2 <= y1: continue

            count += 1

            w_norm = (x2 - x1) / w_img
            h_norm = (y2 - y1) / h_img
            x_center = ((x1 + x2) / 2) / w_img
            y_center = ((y1 + y2) / 2) / h_img
            
            filename = os.path.basename(img_path)
            dst_img = os.path.join(OUTPUT_DIR, 'images', split_name, filename)
            shutil.copy(img_path, dst_img)
            
            txt_name = os.path.splitext(filename)[0] + ".txt"
            dst_label = os.path.join(OUTPUT_DIR, 'labels', split_name, txt_name)
            with open(dst_label, 'w') as f:
                f.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

            if split_name == 'train':
                pad_x = int((x2 - x1) * 0.1)
                pad_y = int((y2 - y1) * 0.1)
                
                x1_z = max(0, x1 - pad_x)
                y1_z = max(0, y1 - pad_y)
                x2_z = min(w_img, x2 + pad_x)
                y2_z = min(h_img, y2 + pad_y)
                
                zoom_img = img[y1_z:y2_z, x1_z:x2_z]
                
                if zoom_img.size > 0:
                    zoom_filename = "zoom_" + filename
                    dst_zoom_img = os.path.join(OUTPUT_DIR, 'images', split_name, zoom_filename)
                    cv2.imwrite(dst_zoom_img, zoom_img)
                    
                    h_z, w_z = zoom_img.shape[:2]
                    
                    plate_w_in_crop = x2 - x1
                    plate_h_in_crop = y2 - y1
                    plate_x_in_crop = (x1 - x1_z) + plate_w_in_crop / 2
                    plate_y_in_crop = (y1 - y1_z) + plate_h_in_crop / 2
                    

                    z_x_center = plate_x_in_crop / w_z
                    z_y_center = plate_y_in_crop / h_z
                    z_w_norm = plate_w_in_crop / w_z
                    z_h_norm = plate_h_in_crop / h_z

                    dst_zoom_label = os.path.join(OUTPUT_DIR, 'labels', split_name, "zoom_" + txt_name)
                    with open(dst_zoom_label, 'w') as f:
                        f.write(f"0 {z_x_center:.6f} {z_y_center:.6f} {z_w_norm:.6f} {z_h_norm:.6f}")
        
        print(f"  -> Imágenes procesadas en {split_name}: {count}")

    process_data_with_zoom(train_df, 'train')
    process_data_with_zoom(val_df, 'val')

    yaml_content = f"""
    path: {os.path.abspath(OUTPUT_DIR)}
    train: images/train
    val: images/val
    nc: 1
    names: ['license_plate']
    """
    with open(os.path.join(OUTPUT_DIR, 'data.yaml'), 'w') as f:
        f.write(yaml_content)

    print("\n Dataset preparado con Zoom Augmentation.")
    print(f"Total imágenes entrenamiento: {len(os.listdir(os.path.join(OUTPUT_DIR, 'images/train')))}")


Procesando train (Generando Zoom Artificial)...


100%|██████████| 662/662 [00:06<00:00, 102.45it/s]


  -> Imágenes procesadas en train: 662
Procesando val (Generando Zoom Artificial)...


100%|██████████| 166/166 [00:01<00:00, 126.70it/s]

  -> Imágenes procesadas en val: 166

 Dataset preparado con Zoom Augmentation.
Total imágenes entrenamiento: 1324
